In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages
import os
import pickle
from tensorly.decomposition import non_negative_parafac

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import seaborn as sns


In [9]:
session_folder = os.listdir("/media/ubuntu/sda/neuropixels/output_dir")
session_list = []
for session in session_folder:
    if 'session_' in session:
        session_list.append(session)

In [12]:
cluster_inf_all = pd.DataFrame()
activity_dict = {}

for session in session_list:
    output_dir = f'/media/ubuntu/sda/neuropixels/output_dir/{session}'
    cluster_inf = pd.read_csv(f'{output_dir}/cluster_inf.csv')
    spike_inf = pd.read_csv(f'{output_dir}/spike_inf.csv', index_col=0)
    stimulus_table = pd.read_csv(f'{output_dir}/stimulus_table.csv')
    stimulus_table = stimulus_table[stimulus_table['frame']!= -1.0]
    stimulus_table['frame'] = stimulus_table['frame'].astype(int)

    filtered_spikes = spike_inf['id'].value_counts()
    filtered_spikes = filtered_spikes[filtered_spikes > 30000].index
    cluster_inf = cluster_inf[cluster_inf['unit_id'].isin(filtered_spikes)]
    spike_inf = spike_inf[spike_inf['id'].isin(filtered_spikes)]

    cluster_inf_all = pd.concat((cluster_inf_all, cluster_inf), axis = 0)

    for neuron in cluster_inf['unit_id']:
        spike_inf_temp = spike_inf[spike_inf['id'] == neuron]
        neuron = f'{neuron}_{session}'
        activity_dict[neuron] = {}
        for image in range(0, 118):
            activity_dict[neuron][image] = []

            trigger_time_temp = stimulus_table[(stimulus_table['frame'] == image)]

            for index, row in trigger_time_temp.iterrows():
                start = row['start_time']
                end = row['stop_time']

                filtered_spikes = spike_inf_temp[(spike_inf_temp['time'] >= start) &
                                                (spike_inf_temp['time'] <= end)]
                if not filtered_spikes.empty:
                    relative_spikes = (filtered_spikes['time'] - start) * 1000
                    activity_dict[neuron][image].append(np.array(relative_spikes))

In [13]:
cluster_inf_all.to_csv("/media/ubuntu/sda/neuropixels/paper_architecture/results/cluster_inf.csv")

In [15]:
with open("/media/ubuntu/sda/neuropixels/paper_architecture/results/activity_dict_30000.pkl", 'wb') as f:
    pickle.dump(activity_dict, f)

In [16]:
activity_dict_region = {}
filtered_neuron = {}
for region in ['VISp', 'VISal', 'VISam', 'VISrl', 'VISpm', 'VISl']:
    activity_dict_region[region] = {}
    cluster_inf_temp = cluster_inf_all[cluster_inf_all['ecephys_structure_acronym'] == region]
    filtered_neuron[region] = cluster_inf_temp['unit_id'].values

    for key, item in activity_dict.items():
        key = key.split("_")[0]
        if int(key) in filtered_neuron[region]:
            activity_dict_region[region][key] = item


In [17]:
import neo
from quantities import ms
from elephant.statistics import instantaneous_rate
from elephant.kernels import GaussianKernel

gk = GaussianKernel(25 * ms)


activity_dict_region_new = {}
for neuron in activity_dict_region['VISp'].keys():
    activity_dict_region_new[neuron] = {}
    for image in range(118):
        temp = []
        for i in activity_dict_region['VISp'][neuron][image]:
            if len(i) > 5:
                i = i.astype(int)
                i = i[i< 250]
                temp_spiketrain = neo.SpikeTrain(list(i) * ms, t_stop=250, t_start=0)
                inst_rate = instantaneous_rate(temp_spiketrain, kernel=gk, sampling_period=10*ms).magnitude

                temp.append(inst_rate.flatten())
            del i
        activity_dict_region_new[neuron][image] = temp

